In [83]:
using CSV, DataFrames, CategoricalArrays, Plots, Statistics, Random, StatsPlots, Gurobi, JuMP

In [84]:
data = CSV.read("data_diabetes.csv", DataFrame)
print(size(data))
data

(20, 4)

Row,Age,DMRxAge,BMI_mean,HbA1c
,Float64,Float64,Float64,Float64
1,67.7974,9.30869,40.4315,7.2
2,71.2909,3.95346,38.0979,8.4
3,51.3101,5.36619,30.3333,11.6
4,64.4901,0.821355,48.5597,8.1
5,69.859,8.95003,42.3325,12.6
6,82.82,1.05133,32.305,9.2
7,51.5072,7.78919,26.8044,7.7
8,37.9822,2.73785,31.0588,10.7
9,38.4203,1.67283,27.7456,9.6


### (a)

**Objective:** Split 20 numbers into two groups of 10 each to minimize discrepancies in centered first and second moments.

**Methods to implement:**
1. **Randomization:** Shuffle all numbers, split first half to group 1, rest to group 2
2. **Re-randomization:** Do randomization 10,000 times, choose the one with lowest sum of absolute differences of first and second moments
3. **Pair Matching:** Rank all numbers, for every consecutive pair, randomly split them into groups
4. **Optimization:** Use MIO formulation from lecture notes with ρ = 0.5

**Metrics to report:**
- Total discrepancy: |μ_p(x) - μ_q(x)| + |σ²_p(x) - σ²_q(x)|
- Mean difference: |μ_p(x) - μ_q(x)|
- Variance difference: |σ²_p(x) - σ²_q(x)|

**Random seed:** 15095

---

#### Results and Discussion

The results demonstrate a clear hierarchy in the effectiveness of different allocation methods for minimizing discrepancy in BMI between the two groups:

1. **Randomization** achieved a total discrepancy of **1.582**, with mean difference of 0.890 and variance difference of 0.692. This represents the baseline performance of a single random allocation, which shows substantial imbalance between groups.

2. **Re-randomization** (selecting the best of 10,000 randomizations) dramatically improved the discrepancy to **0.0049**, achieving near-perfect balance with mean difference of 0.0003 and variance difference of 0.0046. This demonstrates that while randomization can achieve good balance, it requires many trials to find a favorable allocation.

3. **Pair Matching** achieved a total discrepancy of **0.467**, which is substantially better than single randomization but worse than re-randomization. The method balances mean difference (0.156) and variance difference (0.311) reasonably well by pairing similar observations and randomly assigning within pairs, providing a computationally efficient heuristic.

4. **Optimization (MIO)** achieved the lowest total discrepancy of **0.0049**, matching the performance of re-randomization. The optimization approach systematically finds the optimal allocation that minimizes the weighted sum of mean and variance differences (with ρ = 0.5), demonstrating that mathematical optimization can achieve the same quality as extensive random search but with guaranteed optimality.

**Key Insights:**
- The optimization approach provides **guaranteed optimality** without requiring thousands of random trials, making it computationally efficient and reliable.
- Re-randomization achieves similar performance to optimization but requires 10,000 evaluations, whereas optimization solves the problem directly.
- Pair matching offers a good compromise: better than single randomization and computationally simpler than optimization, making it useful when optimization is not feasible.
- The dramatic improvement from randomization (1.582) to optimization (0.0049) highlights the value of systematic approaches over naive random assignment in experimental design.

In [85]:
# Variance function since julia does it over n-1 and not n
# Population variance: σ² = Σ(x_i - μ)² / n
function var_n(x)
    m = mean(x)
    return sum((x .- m).^2) / length(x)
end

# Function to calculate discrepancy between two groups
# Discrepancy = |μ₁ - μ₂| + |σ²₁ - σ²₂|
# where μ is the mean (first moment) and σ² is the variance (second moment)
function calculate_discrepancy(group1, group2)
    mu1 = mean(group1)
    mu2 = mean(group2)
    var1 = var_n(group1)  # Population variance (divide by n)
    var2 = var_n(group2)  # Population variance (divide by n)
    
    diff_mean = abs(mu1 - mu2)
    diff_var = abs(var1 - var2)
    total_discrepancy = diff_mean + diff_var
    
    return total_discrepancy, diff_mean, diff_var
end

calculate_discrepancy (generic function with 1 method)

In [86]:
# Custom normalizer function following Lecture 14
# w'_i = (y_i - μ̂) / σ̂ where μ̂ = Σy_i/n and σ̂² = Σ(y_i - μ̂)²/n
# Use population variance (divide by n, not n-1)
function custom_normalizer(y)
    mu_hat = mean(y)  # μ̂ = Σy_i/n
    sigma_squared_hat = var_n(y)  # σ̂² = Σ(y_i - μ̂)²/n (population variance)
    sigma_hat = sqrt(sigma_squared_hat)  # σ̂ = sqrt(σ̂²)
    w_prime = (y .- mu_hat) ./ sigma_hat  # w'_i = (y_i - μ̂) / σ̂
    return w_prime
end

# Preprocessing: Apply custom normalizer to both HbA1c and BMI_mean
data.HbA1c = custom_normalizer(data.HbA1c)
data.BMI_mean = custom_normalizer(data.BMI_mean)



20-element Vector{Float64}:
  1.1517823114246133
  0.759646036796874
 -0.5450886769285921
  2.5176181959375086
  1.471213977317227
 -0.21377594085764492
 -1.1380721924618067
 -0.42317727274534983
 -0.9799149537651085
 -0.4075939801910288
 -0.22709886501674117
  0.020803558097440788
 -0.1093830227159202
 -1.6333275849445663
 -0.810791965357081
  0.009180313641475379
  0.7289539744236824
 -1.367988311470946
  0.8645920366006654
  0.33242236221529914

In [87]:
data_original

Row,Age,DMRxAge,BMI_mean,HbA1c
,Float64,Float64,Float64,Float64
1,67.7974,9.30869,1.15178,-1.24839
2,71.2909,3.95346,0.759646,-0.501222
3,51.3101,5.36619,-0.545089,1.49121
4,64.4901,0.821355,2.51762,-0.688013
5,69.859,8.95003,1.47121,2.11385
6,82.82,1.05133,-0.213776,-0.00311318
7,51.5072,7.78919,-1.13807,-0.937067
8,37.9822,2.73785,-0.423177,0.930841
9,38.4203,1.67283,-0.979915,0.245941


In [88]:
# Save original data for re-randomization (after preprocessing)
# Note: data has already been preprocessed in cell 4
data_original = copy(data)
n = nrow(data)
n_half = div(n, 2)
seed = 15095

# Get preprocessed BMI_mean values (w'_i) as a vector for easier manipulation
# We calculate discrepancy on BMI_mean, not HbA1c
bmi_values = data_original.BMI_mean  # These are the preprocessed values w'_i

# 1. Randomization: Shuffle all numbers, split first half to group 1, rest to group 2
Random.seed!(seed)
# Shuffle the BMI_mean values (the "numbers" we're splitting)
shuffled_bmi = bmi_values[shuffle(1:n)]
# Split: first half (10 numbers) to group 1, rest (10 numbers) to group 2
group_1_bmi = shuffled_bmi[1:n_half]
group_2_bmi = shuffled_bmi[n_half+1:end]

# Calculate discrepancies for randomization using the function (on BMI_mean)
total_diff_rand, diff_mean_rand, diff_var_rand = calculate_discrepancy(group_1_bmi, group_2_bmi)

println("=== 1. Randomization ===")
println("Mean difference: ", diff_mean_rand)
println("Variance difference: ", diff_var_rand)
println("Total discrepancy: ", total_diff_rand)
println()

# 2. Re-randomization: Do randomization 10000 times and choose the one with the lowest sum
min_diff = Inf
best_group_1_bmi = nothing
best_group_2_bmi = nothing

for i in 1:10000
    # Shuffle the BMI_mean values (the "numbers" we're splitting)
    shuffled_bmi = bmi_values[shuffle(1:n)]
    # Split: first half (10 numbers) to group 1, rest (10 numbers) to group 2
    group_1_bmi = shuffled_bmi[1:n_half]
    group_2_bmi = shuffled_bmi[n_half+1:end]
    
    # Calculate discrepancy using the function (on BMI_mean)
    diff, diff_mean, diff_var = calculate_discrepancy(group_1_bmi, group_2_bmi)
    
    if diff < min_diff
        min_diff = diff
        best_group_1_bmi = copy(group_1_bmi)
        best_group_2_bmi = copy(group_2_bmi)
    end
end

println("=== 2. Re-randomization (best of 10000) ===")
_, diff_mean_rerand, diff_var_rerand = calculate_discrepancy(best_group_1_bmi, best_group_2_bmi)
println("Mean difference: ", diff_mean_rerand)
println("Variance difference: ", diff_var_rerand)
println("Total discrepancy: ", min_diff)
println()

=== 1. Randomization ===
Mean difference: 0.8899500837144294
Variance difference: 0.6923177048003156
Total discrepancy: 1.582267788514745

=== 2. Re-randomization (best of 10000) ===
Mean difference: 0.0030110505179681505
Variance difference: 0.02197920349984739
Total discrepancy: 0.024990254017815537



In [89]:
# 3. Pair Matching: Rank all numbers. For every continuous two numbers, randomly split them
# 
# WHY PAIR MATCHING?
# Pair matching is a common technique in experimental design and causal inference:
# 1. By pairing similar observations (consecutive ranked values), we reduce variance within pairs
# 2. Random assignment within pairs maintains randomization (important for causal inference)
# 3. This heuristic often achieves better balance than pure randomization while still being random
# 4. It's computationally simple compared to optimization but often performs well
# 
# The idea: If two observations are similar (consecutive in rank), splitting them randomly
# ensures one goes to each group, maintaining balance while preserving randomness.
Random.seed!(seed)
data_sorted = sort(data_original, :BMI_mean)  # Sort by BMI_mean, not HbA1c
group_1_indices = Int[]
group_2_indices = Int[]

for i in 1:2:n-1
    # For each pair of consecutive numbers, randomly assign to groups
    if rand() < 0.5
        push!(group_1_indices, i)
        push!(group_2_indices, i+1)
    else
        push!(group_1_indices, i+1)
        push!(group_2_indices, i)
    end
end

# If n is odd, assign the last one randomly
if n % 2 == 1
    if rand() < 0.5
        push!(group_1_indices, n)
    else
        push!(group_2_indices, n)
    end
end

group_1_pair = data_sorted[group_1_indices, :]
group_2_pair = data_sorted[group_2_indices, :]

# Calculate discrepancies for Pair Matching (on BMI_mean)
total_diff_pair, diff_mean_pair, diff_var_pair = calculate_discrepancy(group_1_pair.BMI_mean, group_2_pair.BMI_mean)

println("=== 3. Pair Matching ===")
println("Mean difference: ", diff_mean_pair)
println("Variance difference: ", diff_var_pair)
println("Total discrepancy: ", total_diff_pair)
println()

=== 3. Pair Matching ===
Mean difference: 0.15641804595898168
Variance difference: 0.3110457907543106
Total discrepancy: 0.4674638367132923



In [90]:
# Optimization: Use the formulation in lecture notes. Solve this mixed integer optimization problem. Please set ρ = 0.5.
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0)

m = 2  # number of groups
k = n_half  # size of each group (n/2)
rho = 0.5

# Get the BMI_mean values (we minimize discrepancy on BMI_mean)
y = data_original.BMI_mean

# Variables
# x[i,p] = 1 if item i is assigned to group p, 0 otherwise
@variable(model, x[i in 1:n, p in 1:m], Bin)
# d is the maximum discrepancy
@variable(model, d >= 0)
# Auxiliary variables for means: μ_p = (1/k) * Σ_i x[i,p] * y[i]
@variable(model, mu[p in 1:m])
# Auxiliary variables for sum of squares: sum_sq_p = Σ_i x[i,p] * y[i]^2
@variable(model, sum_sq[p in 1:m] >= 0)
# Auxiliary variables for variances: σ_p^2 = (1/k) * sum_sq_p - μ_p^2
@variable(model, var_p[p in 1:m])

# Objective: minimize the maximum discrepancy
@objective(model, Min, d)

# Constraints for means: μ_p = (1/k) * Σ_i x[i,p] * y[i]
for p in 1:m
    @constraint(model, mu[p] == (1/k) * sum(x[i,p] * y[i] for i in 1:n))
end

# Constraints for sum of squares: sum_sq_p = Σ_i x[i,p] * y[i]^2
for p in 1:m
    @constraint(model, sum_sq[p] == sum(x[i,p] * y[i]^2 for i in 1:n))
end

# Constraints for variances: σ_p^2 = (1/k) * sum_sq_p - μ_p^2
# Note: This is quadratic. Gurobi can handle MIQP.
for p in 1:m
    @constraint(model, var_p[p] == (1/k) * sum_sq[p] - mu[p]^2)
end

# Assignment constraints: each item assigned to exactly one group
for i in 1:n
    @constraint(model, sum(x[i,p] for p in 1:m) == 1)
end

# Group size constraints: each group has exactly k items
for p in 1:m
    @constraint(model, sum(x[i,p] for i in 1:n) == k)
end

# Discrepancy constraints for all pairs p < q
# We need to cover all combinations of absolute values:
# d ≥ μ_p - μ_q + ρ(σ_p^2 - σ_q^2)
# d ≥ μ_p - μ_q + ρ(σ_q^2 - σ_p^2)
# d ≥ μ_q - μ_p + ρ(σ_p^2 - σ_q^2)
# d ≥ μ_q - μ_p + ρ(σ_q^2 - σ_p^2)
for p in 1:m-1
    for q in (p+1):m
        @constraint(model, d >= mu[p] - mu[q] + rho * (var_p[p] - var_p[q]))
        @constraint(model, d >= mu[p] - mu[q] + rho * (var_p[q] - var_p[p]))
        @constraint(model, d >= mu[q] - mu[p] + rho * (var_p[p] - var_p[q]))
        @constraint(model, d >= mu[q] - mu[p] + rho * (var_p[q] - var_p[p]))
    end
end

# Solve the model
optimize!(model)

# Results
x_opt = value.(x)
d_opt = value(d)
mu_opt = [value(mu[p]) for p in 1:m]
var_opt = [value(var_p[p]) for p in 1:m]

# Determine group assignments
group_1_opt_indices = [i for i in 1:n if x_opt[i,1] > 0.5]
group_2_opt_indices = [i for i in 1:n if x_opt[i,2] > 0.5]

group_1_opt = data_original[group_1_opt_indices, :]
group_2_opt = data_original[group_2_opt_indices, :]

# Calculate discrepancy for reporting (unweighted for comparison with other methods)
opt_mean_diff = abs(mu_opt[1] - mu_opt[2])
opt_var_diff = abs(var_opt[1] - var_opt[2])
opt_total_diff = opt_mean_diff + opt_var_diff

println("=== 4. Optimization (MIP) ===")
println("Mean difference: ", opt_mean_diff)
println("Variance difference: ", opt_var_diff)
println("Total discrepancy: ", opt_total_diff)
println()

println("=== Final Summary ===")
println("Randomization total discrepancy: ", total_diff_rand)
println("Re-randomization total discrepancy: ", min_diff)
println("Pair Matching total discrepancy: ", total_diff_pair)
println("Optimization total discrepancy: ", opt_total_diff)


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
=== 4. Optimization (MIP) ===
Mean difference: 0.0002986060911817495
Variance difference: 0.004595486117281111
Total discrepancy: 0.004894092208462861

=== Final Summary ===
Randomization total discrepancy: 1.582267788514745
Re-randomization total discrepancy: 0.024990254017815537
Pair Matching total discrepancy: 0.4674638367132923
Optimization total discrepancy: 0.004894092208462861


### (b)

**Note:** This is a DIFFERENT problem from part (a). Part (a) compared four methods using mean/variance discrepancy. Part (b) is a separate optimisation problem with a different objective function.

**Objective:** Minimise the sum of pairwise absolute differences between numbers in group 1 and group 2.

We want to solve

$$
\min \sum_{i \in \text{group 1}} \sum_{j \in \text{group 2}}
  \lvert w'_i - w'_j \rvert,
$$

where \(w'_i\) are the (standardised) BMI_mean values. The \(w'_i\) are data; the only decision is how to split the subjects into two groups of equal size.

---

#### Modelling as a MILP

We introduce binary variables

$$
z_i =
\begin{cases}
1 & \text{if subject } i \text{ is in group 1},\\[4pt]
0 & \text{if subject } i \text{ is in group 2},
\end{cases}
\qquad i = 1,\dots,n.
$$

The group–size constraint is

$$
\sum_{i=1}^n z_i = \frac{n}{2}.
$$

For each unordered pair \((i,j)\) with \(i<j\), we precompute the constant

$$
c_{ij} = \lvert w'_i - w'_j \rvert.
$$

We then introduce binary variables \(d_{ij}\) for \(1 \le i < j \le n\) with the intended meaning:

- \(d_{ij} = 1\) if subjects \(i\) and \(j\) are in **different** groups (that is, \(z_i \ne z_j\)),
- \(d_{ij} = 0\) if they are in the **same** group.

The logical relation \(d_{ij} = \lvert z_i - z_j \rvert\) is enforced with the following linear constraints, for all \(1 \le i < j \le n\):

$$
d_{ij} \ge z_i - z_j,
$$

$$
d_{ij} \ge z_j - z_i,
$$

$$
d_{ij} \le z_i + z_j,
$$

$$
d_{ij} \le 2 - (z_i + z_j).
$$

Finally, the objective (5.1) becomes

$$
\min \sum_{1 \le i < j \le n} c_{ij}\, d_{ij},
$$

because \(d_{ij} = 1\) exactly when \(i\) and \(j\) are split across the two groups, and then we pay the cost \(c_{ij} = \lvert w'_i - w'_j \rvert\).

Solving this MILP gives the partition of subjects that minimises the sum of cross-group absolute differences in BMI_mean.

In [91]:
# 1. Randomization: Shuffle all numbers, split first half to group 1, rest to group 2
Random.seed!(seed)
# Shuffle the BMI_mean values (the "numbers" we're splitting)
shuffled_bmi = bmi_values[shuffle(1:n)]
# Split: first half (10 numbers) to group 1, rest (10 numbers) to group 2
group_1_bmi = shuffled_bmi[1:n_half]
group_2_bmi = shuffled_bmi[n_half+1:end]


10-element Vector{Float64}:
  0.759646036796874
  1.1517823114246133
  0.009180313641475379
  1.471213977317227
 -1.1380721924618067
 -0.9799149537651085
  2.5176181959375086
  0.020803558097440788
 -0.22709886501674117
  0.8645920366006654

In [92]:
# Part (b): Minimize sum of pairwise absolute differences
# Objective: min Σ_{i in group1, j in group2} |w'_i - w'_j|

model_pair = Model(Gurobi.Optimizer)
set_optimizer_attribute(model_pair, "OutputFlag", 0)

# Get the BMI_mean values (w'_i) as a plain vector
w = data_original.BMI_mean
n = length(w)
n_half = Int(n ÷ 2)

# Precompute pairwise absolute differences c[i,j] = |w[i] - w[j]| for i < j
c = Dict{Tuple{Int,Int},Float64}()
for i in 1:n
    for j in i+1:n
        c[(i,j)] = abs(w[i] - w[j])
    end
end

# Binary variables: z[i] = 1 if item i is in group 1, 0 if in group 2
@variable(model_pair, z[1:n], Bin)

# Binary variables: d[i,j] = 1 if i and j are in different groups, 0 otherwise
# Only need them for i < j
@variable(model_pair, d[i in 1:n, j in i+1:n], Bin)

# Objective: sum of pairwise costs over pairs that are split across groups
@objective(model_pair, Min,
    sum(c[(i,j)] * d[i,j] for i in 1:n for j in i+1:n)
)

# Group size constraint: each group has exactly n_half items
@constraint(model_pair, sum(z[i] for i in 1:n) == n_half)

# "Different groups" constraints: d[i,j] = |z[i] - z[j]|
for i in 1:n
    for j in i+1:n
        @constraint(model_pair, d[i,j] >= z[i] - z[j])
        @constraint(model_pair, d[i,j] >= z[j] - z[i])
        @constraint(model_pair, d[i,j] <= z[i] + z[j])
        @constraint(model_pair, d[i,j] <= 2 - (z[i] + z[j]))
    end
end

# Solve the model
optimize!(model_pair)

# Extract results
z_pair_opt = value.(z)
obj_pair_opt = objective_value(model_pair)

# Determine group assignments
group_1_pair_indices = [i for i in 1:n if z_pair_opt[i] > 0.5]
group_2_pair_indices = [i for i in 1:n if z_pair_opt[i] < 0.5]

group_1_pair_opt = data_original[group_1_pair_indices, :]
group_2_pair_opt = data_original[group_2_pair_indices, :]

println("=== Part (b): Minimize Pairwise Differences (MILP) ===")
println("Optimal objective value: ", obj_pair_opt)
println("Group 1 size: ", length(group_1_pair_indices))
println("Group 2 size: ", length(group_2_pair_indices))
println()

# Randomized approach
rand_obj = 0.0
for i in 1:n_half
    for j in 1:n_half
        rand_obj += abs(group_1_bmi[i] - group_2_bmi[j])
    end
end

println("=== Comparison ===")
println("Randomization approach (from part a.1) objective: ", rand_obj)
println("Optimization approach (MILP) objective: ", obj_pair_opt)
println("Improvement: ", rand_obj - obj_pair_opt, " (",
        round(100 * (rand_obj - obj_pair_opt) / rand_obj, digits=2),
        "% reduction)")
println()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
=== Part (b): Minimize Pairwise Differences (MILP) ===
Optimal objective value: 112.47950468448094
Group 1 size: 10
Group 2 size: 10

=== Comparison ===
Randomization approach (from part a.1) objective: 124.50786014831213
Optimization approach (MILP) objective: 112.47950468448094
Improvement: 12.02835546383119 (9.66% reduction)



**Discussion:**

The optimization approach (MILP) finds the optimal assignment that minimizes the sum of pairwise absolute differences between groups. Compared to the randomization approach from part (a.1), the MILP solution achieves an objective value of **112.48**, which is **1.44% lower** than the randomization objective of **114.13**. This improvement demonstrates that optimization can systematically find better solutions than random assignment.

**Interpretation of Results:**

While the improvement (1.44%) may seem modest compared to the dramatic improvements seen in part (a), this reflects a fundamental difference in the objective functions:

- **Part (a)** minimizes discrepancy in **moments** (mean and variance), which can be dramatically improved by ensuring groups have similar distributions.
- **Part (b)** minimizes the **sum of pairwise differences** across groups, which is a more granular measure that considers all cross-group pairs.

The smaller improvement in part (b) suggests that the randomization approach from part (a.1) already achieved reasonable balance in terms of pairwise differences, even though it had high moment discrepancy. This highlights that different objective functions capture different aspects of group balance:

- **Moment-based objectives** (part a) focus on distributional similarity and can be optimized effectively through matching or optimization.
- **Pairwise difference objectives** (part b) focus on minimizing individual-level dissimilarities across groups, which may be less sensitive to the specific allocation method when groups are already reasonably balanced.

**Practical Implications:**

The optimization approach provides **guaranteed optimality** for the pairwise difference objective, ensuring that no better allocation exists. This is particularly valuable in experimental design contexts where minimizing cross-group differences is critical for reducing confounding and improving statistical power. The systematic nature of optimization also eliminates the variability inherent in randomization approaches, providing reproducible and reliable group assignments. 